# Описание задачи
## В задачи пректа входит:
- Нахождение данных для обученяи модели: парсинг и/или иные источники;
- Предобработка полученных данных
- Создание новых признаков с использованием полученных данных;
- Отбор признаков;
- Отбор моделей;
- Обучение и тестирование отобранных моделей на отобранных признаках;
- Подбор макропараметров модели на отобранных признаках;
- Получение предсказанных моделью значений, подготовка и отправка submission

# Импорт библиотек, установка параметров, определение функций
## Импорт бибилиотек

In [29]:
import pandas as pd
import numpy as np
import json 

from pprint import pprint


## Установка параметров

In [4]:
%pylab inline

pd.set_option('display.max_rows', 70)    # выведем больше строк
pd.set_option('display.max_columns', 30) # выведем больше колонок

Populating the interactive namespace from numpy and matplotlib


## Определение функций

In [40]:
def describe_df(df):
    '''
    Вывод простой статистки DataFrame
    '''
    desc = df.describe(include='all',percentiles=[0.5]).T
    desc['type'] = [type(x) for x in df.iloc[0]]
    desc['NaN'] = df.isna().sum()
    desc.unique = df.nunique()
    desc.top = df.mode(axis=0).iloc[0]
    desc.freq = [df[col].value_counts().iloc[0] for col in  df.columns]
    
    print(desc)
#     return desc

# Изучение  данных для предсказания

Что бы понимать какие признаки нам нужны для обучения модели нам необходимо понять какие признаки описывают наблюдения, для которых мы должны предсказать целевую переменную.  

Поэтому для начала исследуем данные эти наблюдения описывающие.

In [44]:
tt = df_describe(df_test)

In [58]:
tt[(tt.unique == 2)].index

Index(['vendor', 'ПТС', 'Руль'], dtype='object')

In [43]:
df_test = pd.read_csv('./Project_5_data/test.csv')
df_test.info(verbose=False)
df_describe(df_test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Columns: 32 entries, bodyType to Таможня
dtypes: int64(6), object(26)
memory usage: 8.5+ MB


,count,unique,top,freq,mean,std,min,50%,max,type,NaN
bodyType,34686,24,седан,13147,NaN,NaN,NaN,NaN,NaN,<class 'str'>,0
brand,34686,12,BMW,4473,NaN,NaN,NaN,NaN,NaN,<class 'str'>,0
car_url,34686,34686,https://auto.ru/cars/used/sale/audi/100/101694...,1,NaN,NaN,NaN,NaN,NaN,<class 'str'>,0
color,34686,16,чёрный,11010,NaN,NaN,NaN,NaN,NaN,<class 'str'>,0
complectation_dict,6418,2364,"{""id"":""4562904"",""name"":""Elegance"",""available_o...",51,NaN,NaN,NaN,NaN,NaN,<class 'float'>,28268
description,34686,31732,Выгода до 82 000 руб. при обмене на Ваш автомо...,264,NaN,NaN,NaN,NaN,NaN,<class 'str'>,0
engineDisplacement,34686,55,2.0 LTR,7391,NaN,NaN,NaN,NaN,NaN,<class 'str'>,0
enginePower,34686,315,249 N12,1708,NaN,NaN,NaN,NaN,NaN,<class 'str'>,0
equipment_dict,24690,23705,"{""leather"":true}",108,NaN,NaN,NaN,NaN,NaN,<class 'str'>,9996
fuelType,34686,5,бензин,28601,NaN,NaN,NaN,NaN,NaN,<class 'str'>,0


In [ ]:
bodyType brand color

In [39]:
df_test.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Columns: 32 entries, bodyType to Таможня
dtypes: int64(6), object(26)
memory usage: 8.5+ MB


Всего в исходном валидационном датасете 32 колонки с разными типами данных (6 int, 26 object) и разыми типами признаков (бинарные в колонках `vendor`, `ПТС`, `Руль`), категориальные (например в колонках `bodyType`, `brand`, `color`), количественные (например в `enginePower` или `mileage`). Некоторын колонки содержат словари или  

Колонка `brand` содержит 12 уникальных значений. Это позволяет нам существенно сократить обьем работы по парсингу, ведь релевантными будут только данные по эти 12 брендам, поэтому только по ним и будем собирать информацию.

Колонка `car_url` содержит ссылки на изображения на сайте ***auto.ru***, что как бы намекает, что валидационный набор спарсен с сайта ***auto.ru*** 

Три колонки  `priceCurrency`, `Состояние` и `Таможня` содержат только одно уникальное хначение, они ненесут никакой информации, поэтому исключаем их из рассмотрения.  
 
  
!  
!  

Следует рассмтотеть каждую признак/колонку. 


### bodyType

In [13]:
df_test.bodyType.unique()

array(['лифтбек', 'внедорожник 5 дв.', 'хэтчбек 5 дв.', 'седан',
       'компактвэн', 'универсал 5 дв.', 'пикап одинарная кабина',
       'хэтчбек 3 дв.', 'купе', 'кабриолет', 'минивэн',
       'пикап двойная кабина', 'внедорожник 3 дв.', 'родстер', 'микровэн',
       'седан 2 дв.', 'купе-хардтоп', 'фастбек', 'тарга',
       'внедорожник открытый', 'лимузин', 'пикап полуторная кабина',
       'седан-хардтоп', 'фургон'], dtype=object)

### brand

In [14]:
df_test.brand.unique()

array(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'],
      dtype=object)

In [ ]:
yyhy*=-0879

### car_url

In [ ]:
df_test.car_url

### color

In [ ]:
df_test.color

### complectation_dict

In [35]:
df_test.complectation_dict.head()[2]

'{"id":"20026336","name":"Ambition","available_options":["heated-wash-system","airbag-passenger","lock","door-sill-panel","electro-mirrors","mirrors-heat","cooling-box","computer","seat-transformation","wheel-power","fabric-seats","airbag-side","abs","wheel-leather","climate-control-1","esp","adaptive-light","audiopreparation","ashtray-and-cigarette-lighter","front-centre-armrest","electro-window-back","16-inch-wheels","body-mouldings","condition","airbag-driver","isofix","aux","electro-window-front","light-sensor","hcc","ptf","rain-sensor","tyre-pressure","audiosystem-cd","front-seats-heat","wheel-configuration2","wheel-configuration1","immo","12v-socket","third-rear-headrest"]}'

### description

In [33]:
df_test.description.head()

0    Все автомобили, представленные в продаже, прох...
1    ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...
2    Все автомобили, представленные в продаже, прох...
3    КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...
4    ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...
Name: description, dtype: object

### engineDisplacement

In [ ]:
df_test.engineDisplacement

### enginePower

In [ ]:
df_test.enginePower

### equipment_dict

In [30]:
pprint(df_test.equipment_dict[0])

'{"engine-proof":true,"tinted-glass":true,"airbag-driver":true,"aux":true,"isofix":true,"electro-window-front":true,"ashtray-and-cigarette-lighter":true,"airbag-passenger":true,"computer":true,"high-beam-assist":true,"seat-transformation":true,"isofix-front":true,"wheel-power":true,"alarm":true,"lock":true,"door-sill-panel":true,"fabric-seats":true,"electro-mirrors":true,"airbag-rear-side":true,"electro-window-back":true,"steel-wheels":true,"ptf":true,"16-inch-wheels":true,"rain-sensor":true,"airbag-side":true,"audiosystem-cd":true,"dark-interior":true,"cooling-box":true,"condition":true,"abs":true,"power-child-locks-rear-doors":true,"front-seats-heat":true,"wheel-configuration2":true,"wheel-configuration1":true,"12v-socket":true,"park-assist-r":true}'


### fuelType

In [ ]:
df_test.fuelType

### image

In [ ]:
df_test.image

### mileage

In [ ]:
df_test.mileage

### modelDate

In [ ]:
df_test.modelDate

### model_info

In [ ]:
df_test.model_info

### model_name

In [ ]:
df_test.model_name

### name

In [ ]:
df_test.name

### numberOfDoors

In [ ]:
df_test.numberOfDoors

### parsing_unixtime

In [ ]:
df_test.parsing_unixtime

### priceCurrency

In [ ]:
df_test.priceCurrency

### productionDate

In [ ]:
df_test.productionDate

### sell_id

In [ ]:
df_test.sell_id

### super_gen

In [ ]:
df_test.super_gen

### vehicleConfiguration

In [ ]:
df_test.vehicleConfiguration

### vehicleTransmission

In [ ]:
df_test.vehicleTransmission

### vendor

In [47]:
df_test.vendor.unique()

array(['EUROPEAN', 'JAPANESE'], dtype=object)

In [48]:
df_test.vendor.value_counts()

EUROPEAN    20682
JAPANESE    14004
Name: vendor, dtype: int64

In [56]:
df_test.brand[df_test.vendor == 'JAPANESE'].sample(5)   

14866      INFINITI
27005    MITSUBISHI
27896    MITSUBISHI
28594    MITSUBISHI
5354          HONDA
Name: brand, dtype: object

In [55]:
df_test.brand[df_test.vendor == 'EUROPEAN'].sample(5)   

29265         BMW
3241         AUDI
16530    MERCEDES
4570         AUDI
3681         AUDI
Name: brand, dtype: object

### Владельцы

In [ ]:
df_test.Владельцы

### Владение

In [31]:
df_test.Владение.head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Владение, dtype: object

### ПТС

In [16]:
df_test.ПТС.unique()

array(['Оригинал', 'Дубликат', nan], dtype=object)

In [19]:
df_test.ПТС.value_counts()

Оригинал    30098
Дубликат     4587
Name: ПТС, dtype: int64

### Привод

In [ ]:
df_test.Привод

### Руль

In [ ]:
df_test.Руль

### Состояние

In [32]:
df_test.Состояние.head()

0    Не требует ремонта
1    Не требует ремонта
2    Не требует ремонта
3    Не требует ремонта
4    Не требует ремонта
Name: Состояние, dtype: object

### Таможня

In [ ]:
df_test.Таможня